# Mise en forme des données dans un dataframe

## Création du dataframe

In [2]:
import pandas as pd



In [3]:
dfQuestions = pd.DataFrame(
        {
            "id"  : [],
            "Groupe Parlementaire" : [],
            "Nom du député" : [],
            "Question" : []
        })

In [4]:
cols =  ["id",
            "Groupe Parlementaire",
            "Nom du député",
            "Question"]

## Récupération des données d'une question

#### Imports et fonctions utilitaires

In [5]:
!pip install webdriver-manager
!pip install -q lxml
!pip install bs4

import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=9aca3753a7016bde476c9e453ec16494cfafe43141f26bb8dac3d4dd50873606
  Stored in directory: /Users/iokanaanbelfis-simon/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


#### Ouverture de la Page

In [6]:
def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"

In [7]:
k = 1
source_code = request.urlopen(pageurl(k)).read()
page = bs4.BeautifulSoup(source_code)

#### Récupération d'une question

In [8]:
question0 = page.find("div", {"class" : "question"})
question = question0.find("p")
question = question.text.strip()

#### Récupération de la Rubrique et du Titre de la question

In [9]:
rubrique0 = page.findAll("div", {"class" : "question_col33"})
rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]

#### Récupération du ministère interrogé

In [10]:
ministere0 = page.find("div", {"class" : "question_col50"})
ministere = ministere0.text.strip()[23:]

#### Récupération de la date de la question

In [11]:
date0 = page.find("div", {"class" : "question_publish_date"})
date = date0.find("span")
date = date.text.strip()

#### Récuperation des données du parlementaire

In [12]:
lien0 = page.find("span", {"class" : "question_big_content"})
lien = lien0.find("a").get("href")

In [13]:
source_code_parlementaire = request.urlopen(lien).read()
page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)


In [14]:
groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
groupe = groupe0.text.strip()

nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
nom = nom0.text.strip()

#### Récuperation du ministère auquel la question est posée

In [15]:
question 

"M. Christophe Naegelen interroge M. le ministre de l'agriculture et de la souveraineté alimentaire sur le statut particulier des « patous » communément appelés chiens de berger ainsi que sur les dispositifs mis en place pour permettre la cohabitation entre les randonneurs et les patous. Les chiens de berger protègent les troupeaux et ne sont pas considérés comme divaguant lorsqu'ils se trouvent sans laisse et à distance de leur maître depuis la loi n° 2005-157 du 23 février 2005 qui a modifié l'article L. 211-23 du code rural. Désormais, la loi libère « de toute poursuite pénale pour divagation, l'éleveur ou le propriétaire d'un chien affecté à la protection du troupeau (« patous ») ou à la garde de celui-ci (chien de conduite) ». Des études ont montré que les patous ne sont pas dangereux pour les hommes, ces chiens sont dressés pour dissuader et non pour blesser. Néanmoins, des cas d'agressions sont recensés et une forte part des randonneurs sont inquiets de croiser ces chiens. Des p

#### Ajout de la donnée au dataframe

In [16]:
dfQuestion = pd.DataFrame([[k, groupe, nom, question]], columns = cols)

In [17]:
dfQuestions = pd.concat([dfQuestions, dfQuestion], ignore_index = True)
dfQuestions

,id,Groupe Parlementaire,Nom du député,Question
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...


## Création du Data Frame

In [18]:
import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"

In [19]:
n = 100

In [20]:
dfQuestions = pd.DataFrame(
        {
            "id"  : [],
            "Groupe Parlementaire" : [],
            "Nom du député" : [],
            "Question" : []
        })

for k in range(1, n+1) : 
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    dfQuestion = pd.DataFrame([[k, groupe, nom, question]], columns = cols)
    dfQuestions = pd.concat([dfQuestions, dfQuestion], ignore_index = True)

dfQuestions

,id,Groupe Parlementaire,Nom du député,Question
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...
1,2.0,Les Républicains,M. Pierre Cordier,M. Pierre Cordier appelle l'attention de Mme l...
2,3.0,Démocrate (MoDem et Indépendants),M. Frédéric Petit,M. Frédéric Petit appelle l'attention de M. le...
3,4.0,Renaissance,Mme Marie-Pierre Rixain,Mme Marie-Pierre Rixain appelle l'attention de...
4,5.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...
...,...,...,...,...
95,96.0,Les Républicains,Mme Véronique Louwagie,Mme Véronique Louwagie attire l'attention de M...
96,97.0,La France insoumise - Nouvelle Union Populaire...,M. François Ruffin,M. François Ruffin interroge M. le ministre dé...
97,98.0,Les Républicains,M. Thibault Bazin,M. Thibault Bazin appelle l'attention de M. le...
98,99.0,Renaissance,Mme Véronique Riotton,Mme Véronique Riotton attire l'attention de M....


## Cleaning


In [24]:
!pip install spacy 
!python -m spacy download fr_core_news_sm

     |████████████████████████████████| 6.9 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 878 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 181 kB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 6.0 MB/s eta 0:00:011
     |████████████████████████████████| 381 kB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 493 kB 7.4 MB/s eta 0:00:01
  Using cached typing_extensions-4.9.0-py3-none-any.whl (32 kB)
     |████████████████████████████████| 1.9 MB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.1 MB 10.4 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 4.6 MB/s  eta 0:00:01
  Attempting uninstall: typing-extensions
    Foun

In [25]:
import spacy
nlp = spacy.load('fr_core_news_sm')

def clean_docs(texts, remove_stopwords=True, n_process = 4):
    
    docs = nlp.pipe(texts, 
                    n_process=n_process,
                    disable=['parser', 'ner', 'textcat'])
    stopwords = nlp.Defaults.stop_words

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [tok for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

In [26]:
dfQuestions['Question_clean'] = clean_docs(dfQuestions['Question'])
dfQuestions.head()

,id,Groupe Parlementaire,Nom du député,Question,Question_clean
0,1.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...,m. christophe naegelen interroge m. ministre a...
1,2.0,Les Républicains,M. Pierre Cordier,M. Pierre Cordier appelle l'attention de Mme l...,m. pierre cordier appelle attention mme minist...
2,3.0,Démocrate (MoDem et Indépendants),M. Frédéric Petit,M. Frédéric Petit appelle l'attention de M. le...,m. frédéric petit appelle attention m. ministr...
3,4.0,Renaissance,Mme Marie-Pierre Rixain,Mme Marie-Pierre Rixain appelle l'attention de...,mme marie-pierre rixain appelle attention m. m...
4,5.0,"Libertés, Indépendants, Outre-mer et Territoires",M. Christophe Naegelen,M. Christophe Naegelen interroge M. le ministr...,m. christophe naegelen interroge m. ministre é...


## Création d'un fichier csv (semicolon)

In [27]:
données = open("data.csv", "w")

données.write("id|groupe|nom|ministère_interrogé|rubrique|titre|date|question")

données.close()

In [28]:
données = open("data.csv", "a")

for k in range(1, 501) : 
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    rubrique0 = page.findAll("div", {"class" : "question_col33"})
    rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
    rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]
    
    ministere0 = page.find("div", {"class" : "question_col50"})
    ministere = ministere0.text.strip()[23:]

    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    date0 = page.find("div", {"class" : "question_publish_date"})
    date = date0.find("span")
    date = date.text.strip()
    
    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    données.write("\n" + str(k) + "|" + groupe + "|" + nom +  "|" + ministere + "|" + rubrique + "|" + titre + "|" + date + "|" + question)

données.close()